# Kaggle Miniproject 1

## Team Members:
### Karthik Karnik 
### Kapil Sinha
### Anvita Mishra

In [192]:
import numpy as np
import matplotlib.pyplot as plt


from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, Conv2D, MaxPooling2D, BatchNormalization, Activation
from sklearn import tree
from sklearn import base
from sklearn.ensemble import RandomForestClassifier

In [86]:
with open("training_data.txt") as f:
    words = f.readline().split(' ')
training_data = np.loadtxt("training_data.txt", skiprows = 1)
test_data = np.loadtxt("test_data.txt", skiprows = 1)

In [ ]:
x_train = training_data[:, 1:]
y_train = training_data[:, 0]

In [149]:
x_train = training_data[:16000, 1:]
y_train = training_data[:16000, 0]

x_val = training_data[16000:, 1:]
y_val = training_data[16000:, 0]

In [176]:
x_test = test_data

In [116]:
np.shape(x_train)


(16000, 1000)

In [95]:
np.shape(x_test)

(10000, 1000)

In [203]:
#n_words = Xtest.shape[1]
# define network
model = Sequential()
model.add(Dense(100, input_shape=(1000, ), activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 
#model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(x_train, y_train, epochs=50, verbose=2)


Epoch 1/50
1s - loss: 0.5892 - acc: 0.7211
Epoch 2/50
1s - loss: 0.5033 - acc: 0.8195
Epoch 3/50
1s - loss: 0.4616 - acc: 0.8347
Epoch 4/50
1s - loss: 0.4299 - acc: 0.8459
Epoch 5/50
1s - loss: 0.4066 - acc: 0.8566
Epoch 6/50
1s - loss: 0.3820 - acc: 0.8684
Epoch 7/50
2s - loss: 0.3586 - acc: 0.8794
Epoch 8/50
2s - loss: 0.3379 - acc: 0.8886
Epoch 9/50
1s - loss: 0.3198 - acc: 0.8976
Epoch 10/50
1s - loss: 0.3022 - acc: 0.9051
Epoch 11/50
1s - loss: 0.2897 - acc: 0.9095
Epoch 12/50
1s - loss: 0.2860 - acc: 0.9091
Epoch 13/50
1s - loss: 0.2764 - acc: 0.9141
Epoch 14/50
1s - loss: 0.2640 - acc: 0.9189
Epoch 15/50
1s - loss: 0.2603 - acc: 0.9200
Epoch 16/50
2s - loss: 0.2538 - acc: 0.9220
Epoch 17/50
1s - loss: 0.2474 - acc: 0.9249
Epoch 18/50
1s - loss: 0.2375 - acc: 0.9289
Epoch 19/50
1s - loss: 0.2401 - acc: 0.9268
Epoch 20/50
1s - loss: 0.2284 - acc: 0.9324
Epoch 21/50
1s - loss: 0.2294 - acc: 0.9311
Epoch 22/50
1s - loss: 0.2278 - acc: 0.9318
Epoch 23/50
1s - loss: 0.2179 - acc: 0.93

In [204]:
# evaluate
loss, acc = model.evaluate(x_val, y_val, verbose=0)
print('Validation Accuracy: %f' % (acc*100))

Validation Accuracy: 84.200000


In [181]:
print(x_test)

predictions = model.predict(x_test, verbose=1)


[[ 2.  3.  1. ...,  0.  0.  0.]
 [ 2.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  1. ...,  0.  0.  0.]
 [ 3.  1.  0. ...,  0.  0.  0.]
 [ 1.  1.  1. ...,  0.  0.  0.]]
 9792/10000 [============================>.] - ETA: 0s

In [188]:
#boolList = predictions >= 1
print(predictions)
lst = np.where(predictions >= 0.5, 1, 0)
lst = lst.flatten()
print(lst)

[[ 0.89242476]
 [ 0.89248949]
 [ 0.01895684]
 ..., 
 [ 0.01875839]
 [ 0.89249879]
 [ 0.01878286]]
[1 1 0 ..., 0 1 0]


In [183]:
"""
Takes in a numpy array consisting of outputs for our test set and
outputs a file in the correct submission format (as per the sample
submission file)
"""
def make_submission_file(arr, fname = "submission.txt"):
    f = open(fname, 'w')
    f.write("Id,Prediction\n")
    for i in range(len(arr)):
        f.write(str(i + 1) + "," + str(arr[i]) + '\n')
    f.close()

In [189]:
make_submission_file(lst)

In [148]:
# Random forest model

def classification_err(y, real_y):
    """
    This function returns the classification error between two equally-sized vectors of 
    labels; this is the fraction of samples for which the labels differ.
    
    Inputs:
        y: (N, ) shaped array of predicted labels
        real_y: (N, ) shaped array of true labels
    Output:
        Scalar classification error
    """
    tot_err = 0
    
    # Count how many samples the labels differ between the two vectors
    for i in range(len(y)):
        if y[i] != real_y[i]:
            tot_err += 1
    
    # Get the fraction of samples for which the labels differ
    classification_error = float(tot_err) / len(y)
    
    return classification_error
    
def eval_tree_based_model_max_depth(clf, max_depth, X_train, y_train, X_test, y_test):
    """
    This function evaluates the given classifier (either a decision tree or random forest) at all of the 
    minimum leaf size parameters in the vector min_samples_leaf, using the given training and testing
    data. It returns two vector, with the training and testing classification errors.
    
    Inputs:
        clf: either a decision tree or random forest classifier object
        min_samples_leaf: a (T, ) vector of all the min_samples_leaf stopping condition parameters 
                            to test, where T is the number of parameters to test
        X_train: (N, D) matrix of training samples.
        y_train: (N, ) vector of training labels.
        X_test: (N, D) matrix of test samples
        y_test: (N, ) vector of test labels
    Output:
        train_err: (T, ) vector of classification errors on the training data
        test_err: (T, ) vector of classification errors on the test data
    """
    train_err = []
    val_err = []
    
    # Use minimum leaf size parameters to create, train, and test a classifier

    # Make a new copy of the classifier, which allows us to "forget" about the
    # fitting from the previous min_leaf value
    for depth in max_depth:
        clf_copy = base.clone(clf)
        clf_copy.set_params(max_depth=depth)
        clf_copy.fit(X_train, y_train)

        train_predict = clf_copy.predict(X_train)
        test_predict = clf_copy.predict(X_test)

        train_err.append(classification_err(train_predict, y_train))
        val_err.append(classification_err(test_predict, y_test))

    return train_err, val_err



n_estimators = 100
clf = RandomForestClassifier(n_estimators = n_estimators, criterion = 'gini')

max_depth = np.arange(10, 51)

train_err, test_err = eval_tree_based_model_max_depth(clf, max_depth, x_train, y_train, x_val, y_val)



KeyboardInterrupt: 

In [ ]:
x_train = training_data[:, 1:]
y_train = training_data[:, 0]
x_test = test_data

def random_forest_predict(clf, max_depth, X_train, y_train, X_test):
    """
    This function evaluates the given classifier (either a decision tree or random forest) at all of the 
    minimum leaf size parameters in the vector min_samples_leaf, using the given training and testing
    data. It returns two vector, with the training and testing classification errors.
    
    Inputs:
        clf: either a decision tree or random forest classifier object
        min_samples_leaf: a (T, ) vector of all the min_samples_leaf stopping condition parameters 
                            to test, where T is the number of parameters to test
        X_train: (N, D) matrix of training samples.
        y_train: (N, ) vector of training labels.
        X_test: (N, D) matrix of test samples
        y_test: (N, ) vector of test labels
    Output:
        train_err: (T, ) vector of classification errors on the training data
        test_err: (T, ) vector of classification errors on the test data
    """
    train_err = []
    val_err = []
    
    # Use minimum leaf size parameters to create, train, and test a classifier

    # Make a new copy of the classifier, which allows us to "forget" about the
    # fitting from the previous min_leaf value
    clf_copy = base.clone(clf)
    clf_copy.set_params(max_depth= max_depth)
    clf_copy.fit(X_train, y_train)

    train_predict = clf_copy.predict(X_train)
    test_predict = clf_copy.predict(X_test)

    train_err.append(classification_err(train_predict, y_train))

    return train_err, test_predict

train_err, test_predict = random_forest_predict(clf, 49, x_train, y_train, x_test)

print(train_err)

print(test_predict)

In [144]:
test_predict.astype(int)

array([1, 1, 0, ..., 0, 1, 0])

In [ ]:
plt.figure()
plt.plot(max_depth, test_err, label='Testing error')
plt.plot(max_depth, train_err, label='Training error')
plt.xlabel('Maximum Tree Depth')
plt.ylabel('Validation error')
plt.title('Decision Tree with Gini Impurity and Maximum Tree Depth')
plt.legend(loc=0, shadow=True, fontsize='x-large')
plt.show()

print('Test error minimized at max_depth = %i' % max_depth[np.argmin(test_err)])

In [145]:
make_submission_file(test_predict.astype(int))

In [133]:
print(len(test_predict))

10000
